In [22]:
#Import Libralies
import numpy as np
import pandas as pd
import os
import random as rn
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

print(tf.__version__)

2.5.0


In [23]:
# Function For Result Evaluation 
def metrics(y_true, y_pred):
    metrics = [
        r2_score(y_true=y_true, y_pred=y_pred), 
        np.sqrt(mean_squared_error(y_true=y_true, y_pred=y_pred)), 
        mean_absolute_error(y_true=y_true, y_pred=y_pred)
    ]
    
    return metrics

In [24]:
# Function For Result Visualization
def yyplot(y_true, y_pred):
    yvalues = np.concatenate([y_true.flatten(), y_pred.flatten()])
    ymin, ymax, yrange = np.amin(yvalues), np.amax(yvalues), np.ptp(yvalues)
    fig = plt.figure(figsize=(8, 8))
    plt.scatter(y_true, y_pred, s=1.0)
    plt.plot([ymin - yrange * 0.01, ymax + yrange * 0.01], [ymin - yrange * 0.01, ymax + yrange * 0.01], color="red", linestyle = "dashed")
    plt.grid(linestyle = "dashed")
    plt.xlim(ymin - yrange * 0.01, ymax + yrange * 0.01)
    plt.ylim(ymin - yrange * 0.01, ymax + yrange * 0.01)
    plt.xlabel('y: true value')
    plt.ylabel('y: predicted value')
    plt.title('True-Predicted Plot')
    plt.show()

    return fig

In [25]:
# Function to fix random seed
def reset_seed(seed=0):
    os.environ['PYTHONHASHSEED'] = '0'
    rn.seed(seed) # Fixing the seed of a random function
    np.random.seed(seed) # Fixing the seed of numpy
    tf.random.set_seed(seed) # Fixing the seed of tensorflow

In [26]:
#Data Loading
import glob

folder_path=r"/content/drive/MyDrive/Colab Notebooks/CGH/AID/Dataset"
# folder_path = input("Enter the path of the folder where the data is stored.")
# folder_path = "../../02_Operating_Data/" #CGH Updated
# folder_path example: "C:\Users\AB280\Documents\GitHub\AIO_Regression\Dataset_generate\hysys_output\lng_production"
# # If pkl format, 
# all_files = glob.glob(folder_path + "\*.pkl")
# print(all_files)
# list = []
# for file_name in all_files:
#     df_each = pd.read_pickle(file_name)
#     list.append(df_each)
# df = pd.concat(list, axis=0, ignore_index=True)
# df.head(100)

# If csv format,
all_files = glob.glob(folder_path + "/*.csv") #Beware Backslash and Slash
print(all_files) #To Print File Names

list = []
for file_name in all_files:
    df_each = pd.read_csv(file_name)
    list.append(df_each)
    
df = pd.concat(list, axis=0, ignore_index=True) #To combine all files
df.head(100)

['/content/drive/MyDrive/Colab Notebooks/CGH/AID/Dataset/dslng_operating_data_hourly_R3.csv']


,Unnamed: 0,041AI1806A.PV,041AI1806B.PV,041AI1806C.PV,041AI1806D.PV,041AI1806E.PV,041AI1806F.PV,041AI1806G.PV,041PI1203.PV,041TI1306.PV,041TI1356.PV,041TI1355.PV,041PIC1210.PV,041TI1313.PV,041FIC2430.PV,041FQI1004.PV,041FI1004.PV,041AI1806H.PV,071FI1021A.PV,060TI4001.PV,060TI4002.PV,091TI4002.PV,051TI1401A.PV,051TI1401B.PV,051TI1401C.PV,051TI1401D.PV,051TI1401E.PV,051PI1251.PV,051PFIC2940.MV,051FIC1051.MV,051FI1051A.PV,051FI1051B.PV,051AI1781A.PV,051AI1781B.PV,051AI1781C.PV,051AI1781D.PV,051TI1356.PV,051TI1361.PV,051TI1363.PV,051PI1256.PV,...,051PI1260.PV,051PI3110.PV,051TI3110.PV,051PI3111.PV,051TI3111.PV,051PI3120.PV,051TI3120.PV,051PI3121.PV,051TI3121.PV,051FI3121_N.PV,051SI3203.PV,051SI3113.PV,051SI3403.PV,063FI1001.PV,051FI1081.PV,051FI1083.PV,051FI1085.PV,051XI4055.PV,051XI5055.PV,051PI3010A.PV,051TI3010.PV,051FI3010_N.PV,051PI3020.PV,051TI3020.PV,051FI3020_N.PV,051PI3030.PV,051TI3030.PV,051FI3030_N.PV,051PI3041.PV,051TI3041.PV,051FI3041_N.PV,051TI1307.PV,051TI1311.PV,051TI1351.PV,051TI1352.PV,091TI4003.PV,091TI4004.PV,091TI4005.PV,071FI1006.PV,011PIC1202.PV
0,2018-08-01 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-08-01 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-08-01 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-08-01 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-08-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2018-08-05 07:00:00,0.800612,92.713402,3.596027,1.727260,0.450251,0.566872,479.671615,48.006977,-34.143967,-134.507248,-134.988403,42.551053,-155.881638,685.038056,5233.740443,697.501176,438.553033,-2.684979,30.680669,27.285276,28.260326,26.038241,25.875588,26.462817,26.542207,26.291318,51.775191,34.20005,87.799232,106536.230603,938.022240,6.173804,41.526064,41.466542,10.356207,-33.151028,-33.148998,-134.093613,47.469095,...,2.124195,1.999017,-38.070877,15.803739,67.252678,15.178903,35.144389,53.377907,121.219663,539954.337139,4760.908420,4692.943737,4675.125603,11425.588299,10473.481136,10463.639763,9375.451670,2608.041685,517.022411,0.145150,-38.593472,140352.300210,2.030800,-13.528758,178623.565724,5.061242,7.579920,247921.055375,17.512291,68.871984,566715.190889,50.305453,37.786670,35.413122,41.331781,26.305195,26.491366,26.406933,28516.474213,50.195752
96,2018-

In [27]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18596 entries, 73 to 18668
Data columns (total 86 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      18596 non-null  object 
 1   041AI1806A.PV   18596 non-null  float64
 2   041AI1806B.PV   18596 non-null  float64
 3   041AI1806C.PV   18596 non-null  float64
 4   041AI1806D.PV   18596 non-null  float64
 5   041AI1806E.PV   18596 non-null  float64
 6   041AI1806F.PV   18596 non-null  float64
 7   041AI1806G.PV   18596 non-null  float64
 8   041PI1203.PV    18596 non-null  float64
 9   041TI1306.PV    18596 non-null  float64
 10  041TI1356.PV    18596 non-null  float64
 11  041TI1355.PV    18596 non-null  float64
 12  041PIC1210.PV   18596 non-null  float64
 13  041TI1313.PV    18596 non-null  float64
 14  041FIC2430.PV   18596 non-null  float64
 15  041FQI1004.PV   18596 non-null  float64
 16  041FI1004.PV    18596 non-null  float64
 17  041AI1806H.PV   18596 non-null

In [28]:
dict_X = {
    'Unnamed: 0':'Date',
    '041AI1806A.PV':   'FEED_N2',
    '041AI1806B.PV':   'FEED_C1',
    '041AI1806C.PV':   'FEED_C2',
    '041AI1806D.PV':   'FEED_C3',
    '041AI1806E.PV':   'FEED_iC4',
    '041AI1806F.PV':   'FEED_nC4',
    '041AI1806G.PV':   'FEED_C5+',
    '041PI1203.PV':   'MCHE_P_IN',
    '041TI1306.PV':   'FEED_MCHE_IN', #10
    '041TI1356.PV':   'FEED_MCHE_WB_OUT', 
    '041TI1355.PV':   'FEED_MCHE_WB_OUT_2',
    '041PIC1210.PV':   'MCHE_OUT_P',
    '041TI1313.PV':   'MCHE_OUT',
    '041FIC2430.PV':   'LNG_Rundown',
    '041FQI1004.PV':   'LNG_Rundown_Mol',
    '041FI1004.PV':   'LNG_Rundown_2',
    '041AI1806H.PV':   'LNG_Dens',
    '071FI1021A.PV':   'LNG_Loading',
    '060TI4001.PV':   'AmbTemp_1', #20
    '060TI4002.PV':   'AmbTemp_2', 
    '091TI4002.PV':   'AmbTemp_5',
    '051TI1401A.PV':   'AirTemp_A',
    '051TI1401B.PV':   'AirTemp_B',
    '051TI1401C.PV':   'AirTemp_C',
    '051TI1401D.PV':   'AirTemp_D',
    '051TI1401E.PV':   'AirTemp_E',
    '051PI1251.PV':   'P_HPMR',
    '051PFIC2940.MV':   'COLD_JT',
    '051FIC1051.MV':   'WARM_JT', #30
    '051FI1051A.PV':   'MRV_F', 
    '051FI1051B.PV':   'MRL_F',
    '051AI1781A.PV':   'MR_N2',
    '051AI1781B.PV':   'MR_C1',
    '051AI1781C.PV':   'MR_C2',
    '051AI1781D.PV':   'MR_C3',
    '051TI1356.PV':   'MRV_MCHE_IN_T',
    '051TI1361.PV':   'MRL_MCHE_IN_T',
    '051TI1363.PV':   'MRL_MCHE_WB_OUT_T',
    '051PI1256.PV':   'MRL_MCHE_WB_OUT_P', #40
    '051TI1364.PV':   'MRL_MCHE_WB_IN_T', 
    '051TI1359.PV':   'MRV_MCHE_CB_OUT',
    '051PI1253.PV':   'MRV_MCHE_WB_OUT_P',
    '051PI1254.PV':   'MRV_MCHE_CB_OUT_P',
    '051TI1360.PV':   'MRV_MCHE_CB_IN',
    '051TI1370.PV':   'MR_Return_T',
    '051PI1260.PV':   'MR_Return_P',
    '051PI3110.PV':   'LPMR_Suc_P',
    '051TI3110.PV':   'LPMR_Suc_T',
    '051PI3111.PV':   'LPMR_Dis_P', #50
    '051TI3111.PV':   'LPMR_Dis_T', 
    '051PI3120.PV':   'HPMR_Suc_P',
    '051TI3120.PV':   'HPMR_Suc_T',
    '051PI3121.PV':   'HPMR_Dis_P',
    '051TI3121.PV':   'HPMR_Dis_T',
    '051FI3121_N.PV':   'MR_FLOW',
    '051SI3203.PV':   'C3_GT_Speed',
    '051SI3113.PV':   'LPMR_GT_Speed',
    '051SI3403.PV':   'HPMR_GT_Speed',
    '063FI1001.PV':   'FFF_Flow', #60
    '051FI1081.PV':   'Fuel_Flow_C3_Driver',
    '051FI1083.PV':   'Fuel_Flow_LPMR_Driver',
    '051FI1085.PV':   'Fuel_Flow_HPMR_Driver',
    '051XI4055.PV':   'Helper_Motor_C3',
    '051XI5055.PV':   'Helper_Motor_LPMR',
    '051PI3010A.PV':   'LPC3_Suc_P',
    '051TI3010.PV':   'LPC3_Suc_T',
    '051FI3010_N.PV':   'LPC3_Suc_F',
    '051PI3020.PV':   'MPC3_Suc_P',
    '051TI3020.PV':   'MPC3_Suc_T', #70
    '051FI3020_N.PV':   'MPC3_Suc_F',
    '051PI3030.PV':   'HPC3_Suc_P',
    '051TI3030.PV':   'HPC3_Suc_T',
    '051FI3030_N.PV':   'HPC3_Suc_F',
    '051PI3041.PV':   'C3_Comp_Dis_P',
    '051TI3041.PV':   'C3_Comp_Dis_T',
    '051FI3041_N.PV':   'C3_FLOW',
    '051TI1307.PV':   'C3_CON_OUT',
    '051TI1311.PV':   'C3_SUB_OUT',
    '051TI1351.PV':   'LPMR_OUT', #80
    '051TI1352.PV':   'HPMR_OUT', #81
}

col_names=df.columns.tolist()
new_names=[dict_X.get(each, each) for each in col_names] #ရှိရင် ရှိ မရှိရင်ထည့်
df.columns=new_names

In [29]:
df=df.dropna() #Some Values might be zero and thus result in N/A.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18596 entries, 73 to 18668
Data columns (total 86 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   18596 non-null  object 
 1   FEED_N2                18596 non-null  float64
 2   FEED_C1                18596 non-null  float64
 3   FEED_C2                18596 non-null  float64
 4   FEED_C3                18596 non-null  float64
 5   FEED_iC4               18596 non-null  float64
 6   FEED_nC4               18596 non-null  float64
 7   FEED_C5+               18596 non-null  float64
 8   MCHE_P_IN              18596 non-null  float64
 9   FEED_MCHE_IN           18596 non-null  float64
 10  FEED_MCHE_WB_OUT       18596 non-null  float64
 11  FEED_MCHE_WB_OUT_2     18596 non-null  float64
 12  MCHE_OUT_P             18596 non-null  float64
 13  MCHE_OUT               18596 non-null  float64
 14  LNG_Rundown            18596 non-null  float64
 15  L

In [30]:
#Preparation for SpecPower Calculation (Enthalpy Case)
#As per Note-3, Holding Mode: Loading Rate ≒ 0 and Rundown Rate > 85% * Max
df = df[df["LNG_Loading"]<=0]
df = df[df["LNG_Rundown"]>=(df["LNG_Rundown"].max()*.85)]

#Calculations for LNG Prod & Others
df["LNG_Prod"]=df["LNG_Rundown"] * df["LNG_Dens"]/1000

df["MCHE_BTM_DT"]=df["FEED_MCHE_IN"]-df["MR_Return_T"]

df["Fuel_WtFlow_C3_Driver"]=df["Fuel_Flow_C3_Driver"]  / 22.414 * 17.44

df["C3_GT_Power"]=df["Fuel_WtFlow_C3_Driver"] * 50044 / (26.059873 * df["AmbTemp_2"] - 0.848894 * df["C3_GT_Speed"]  + 15378.1778218)

df["Fuel_WtFlow_LPMR_Driver"]=df["Fuel_Flow_LPMR_Driver"] / 22.414 * 17.44

df["LPMR_GT_Power"]=df["Fuel_WtFlow_LPMR_Driver"] * 50044 / (26.059873 * df["AmbTemp_2"] - 0.848894 * df["LPMR_GT_Speed"]  + 15378.1778218)

df["Fuel_WtFlow_HPMR_Driver"]=df["Fuel_Flow_HPMR_Driver"] / 22.414 * 17.44

df["HPMR_GT_Power"]=df["Fuel_WtFlow_HPMR_Driver"] * 50044 / (26.059873 * df["AmbTemp_2"] - 0.848894 * df["HPMR_GT_Speed"]  + 15378.1778218)

**C4 Addition**

In [31]:
df['MR_C4']=100-df['MR_N2']-df['MR_C1']-df['MR_C2']-df['MR_C3']
df=df[['Date',
 'FEED_N2',
 'FEED_C1',
 'FEED_C2',
 'FEED_C3',
 'FEED_iC4',
 'FEED_nC4',
 'FEED_C5+',
 'MCHE_P_IN',
 'FEED_MCHE_IN',
 'FEED_MCHE_WB_OUT',
 'FEED_MCHE_WB_OUT_2',
 'MCHE_OUT_P',
 'MCHE_OUT',
 'LNG_Rundown',
 'LNG_Rundown_Mol',
 'LNG_Rundown_2',
 'LNG_Dens',
 'LNG_Loading',
 'AmbTemp_1',
 'AmbTemp_2',
 'AmbTemp_5',
 'AirTemp_A',
 'AirTemp_B',
 'AirTemp_C',
 'AirTemp_D',
 'AirTemp_E',
 'P_HPMR',
 'COLD_JT',
 'WARM_JT',
 'MRV_F',
 'MRL_F',
 'MR_N2',
 'MR_C1',
 'MR_C2',
 'MR_C3',
 'MR_C4',
 'MRV_MCHE_IN_T',
 'MRL_MCHE_IN_T',
 'MRL_MCHE_WB_OUT_T',
 'MRL_MCHE_WB_OUT_P',
 'MRL_MCHE_WB_IN_T',
 'MRV_MCHE_CB_OUT',
 'MRV_MCHE_WB_OUT_P',
 'MRV_MCHE_CB_OUT_P',
 'MRV_MCHE_CB_IN',
 'MR_Return_T',
 'MR_Return_P',
 'LPMR_Suc_P',
 'LPMR_Suc_T',
 'LPMR_Dis_P',
 'LPMR_Dis_T',
 'HPMR_Suc_P',
 'HPMR_Suc_T',
 'HPMR_Dis_P',
 'HPMR_Dis_T',
 'MR_FLOW',
 'C3_GT_Speed',
 'LPMR_GT_Speed',
 'HPMR_GT_Speed',
 'FFF_Flow',
 'Fuel_Flow_C3_Driver',
 'Fuel_Flow_LPMR_Driver',
 'Fuel_Flow_HPMR_Driver',
 'Helper_Motor_C3',
 'Helper_Motor_LPMR',
 'LPC3_Suc_P',
 'LPC3_Suc_T',
 'LPC3_Suc_F',
 'MPC3_Suc_P',
 'MPC3_Suc_T',
 'MPC3_Suc_F',
 'HPC3_Suc_P',
 'HPC3_Suc_T',
 'HPC3_Suc_F',
 'C3_Comp_Dis_P',
 'C3_Comp_Dis_T',
 'C3_FLOW',
 'C3_CON_OUT',
 'C3_SUB_OUT',
 'LPMR_OUT',
 'HPMR_OUT',
 'LNG_Prod',
 'MCHE_BTM_DT',
 'Fuel_WtFlow_C3_Driver',
 'C3_GT_Power',
 'Fuel_WtFlow_LPMR_Driver',
 'LPMR_GT_Power',
 'Fuel_WtFlow_HPMR_Driver',
 'HPMR_GT_Power'
 ]]


In [32]:
# Load AI model
from tensorflow.keras.models import Sequential, load_model
model = load_model('/content/drive/MyDrive/Colab Notebooks/CGH/AID/Step2_Enthalpy Calculation/C3_Enthalpy_Model.h5') #CGH's provided Enthalpy Regression Model File

# Load Scaler used for Model
import pickle
sc_X = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/CGH/AID/Step2_Enthalpy Calculation/C3_Enthalpy_Scaler_Input.pkl", "rb")) #This picklet was created with Scikit-Learn 0.23.1
sc_y= pickle.load(open("/content/drive/MyDrive/Colab Notebooks/CGH/AID/Step2_Enthalpy Calculation/C3_Enthalpy_Scaler_Output.pkl","rb"))
sc_X.mean_

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


array([12.29291413, 96.41336579])

In [33]:
# explanatory variables for C3_Enthalpy data
list_X = ['LPC3_Suc_P',
          'LPC3_Suc_T']

#Standardization for inputs
X = df.loc[:, list_X].values
X_std = sc_X.transform(X)

#sc_X.fit(X=X)
X_std.mean(axis=0)

LP_Suc_enthpy_scaled = model.predict(X_std)
LP_Suc_enthpy = sc_y.inverse_transform(LP_Suc_enthpy_scaled)
df['LPC3_Suc_Enthpy']=LP_Suc_enthpy

In [34]:
list_X = ['MPC3_Suc_P',
          'MPC3_Suc_T']
          

X = df.loc[:, list_X].values
X_std = sc_X.transform(X)
X_std.mean(axis=0)

MP_Suc_enthpy_scaled = model.predict(X_std)
MP_Suc_enthpy = sc_y.inverse_transform(MP_Suc_enthpy_scaled)
df['MPC3_Suc_Enthpy']=MP_Suc_enthpy

In [35]:
list_X = ['HPC3_Suc_P',
          'HPC3_Suc_T']
X = df.loc[:, list_X].values
X_std = sc_X.transform(X)
X_std.mean(axis=0)

HP_Suc_enthpy_scaled = model.predict(X_std)
HP_Suc_enthpy = sc_y.inverse_transform(HP_Suc_enthpy_scaled)
df['HPC3_Suc_Enthpy']=HP_Suc_enthpy

In [36]:
list_X = ['C3_Comp_Dis_P',
          'C3_Comp_Dis_T']
X = df.loc[:, list_X].values
X_std = sc_X.transform(X)
X_std.mean(axis=0)

C3_Disc_enthpy_scaled = model.predict(X_std)
C3_Disc_enthpy = sc_y.inverse_transform(C3_Disc_enthpy_scaled)
df['C3_Comp_Dis_Enthpy']=C3_Disc_enthpy

In [37]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/CGH/AID/Step2_Enthalpy Calculation/MR_Enthalpy_Model_C4.h5')

# Load Scaler used for Model
import pickle
sc_X = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/CGH/AID/Step2_Enthalpy Calculation/MR_Enthalpy_C4_Scaler_Input.pkl", "rb"))
sc_y= pickle.load(open("/content/drive/MyDrive/Colab Notebooks/CGH/AID/Step2_Enthalpy Calculation/MR_Enthalpy_C4_Scaler_Output.pkl","rb"))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [38]:
list_X = ['MR_N2',
          'MR_C1',
          'MR_C2',
          'MR_C3',
          'LPMR_Suc_P',
          'LPMR_Suc_T']
X = df.loc[:, list_X].values
X_std = sc_X.transform(X)
X_std.mean(axis=0)

LPMR_Suc_enthpy_scaled = model.predict(X_std)
LPMR_Suc_enthpy = sc_y.inverse_transform(LPMR_Suc_enthpy_scaled)
df['LPMR_Suc_Enthpy']=LPMR_Suc_enthpy
X_std.mean(axis=0)
sc_X.mean_

array([10.51709397, 36.42673198, 36.28899324, 15.83059358, 32.52726607,
       83.12243322])

In [39]:
list_X = ['MR_N2',
          'MR_C1',
          'MR_C2',
           'MR_C3',
          'LPMR_Dis_P',
          'LPMR_Dis_T']
X = df.loc[:, list_X].values
X_std = sc_X.transform(X)
X_std.mean(axis=0)

LPMR_Dis_enthpy_scaled = model.predict(X_std)
LPMR_Dis_enthpy = sc_y.inverse_transform(LPMR_Dis_enthpy_scaled)
df['LPMR_Dis_Enthpy']=LPMR_Dis_enthpy

In [40]:
list_X = ['MR_N2',
          'MR_C1',
          'MR_C2',
           'MR_C3',
          'HPMR_Suc_P',
          'HPMR_Suc_T']
X = df.loc[:, list_X].values
X_std = sc_X.transform(X)
X_std.mean(axis=0)

HPMR_Suc_enthpy_scaled = model.predict(X_std)
HPMR_Suc_enthpy = sc_y.inverse_transform(HPMR_Suc_enthpy_scaled)
df['HPMR_Suc_Enthpy']=HPMR_Suc_enthpy

In [41]:
list_X = ['MR_N2',
          'MR_C1',
          'MR_C2',
           'MR_C3',
          'HPMR_Dis_P',
          'HPMR_Dis_T']
X = df.loc[:, list_X].values
X_std = sc_X.transform(X)
X_std.mean(axis=0)

HPMR_Dis_enthpy_scaled = model.predict(X_std)
HPMR_Dis_enthpy = sc_y.inverse_transform(HPMR_Dis_enthpy_scaled)
df['HPMR_Dis_Enthpy']=HPMR_Dis_enthpy

In [42]:
#Preparation for SpecPower Calculation (Enthalpy Case)
#As per Note-3, Holding Mode: Loading Rate ≒ 0 and Rundown Rate > 85% * Max
# df = df[df["LNG_Loading"]<=0]
# df = df[df["LNG_Rundown"]>=(df["LNG_Rundown"].max()*.85)]

#Calculations for SpecPower Calculation
df['C3_Comp_Power_Suc_Enthpy']=((df['LPC3_Suc_Enthpy']*df['LPC3_Suc_F']) + (df['MPC3_Suc_Enthpy']*df['MPC3_Suc_F']) + (df['HPC3_Suc_Enthpy']*df['HPC3_Suc_F']))/22.414
df['C3_Comp_Power_Dis_Enthpy']= df['C3_Comp_Dis_Enthpy']*df['C3_FLOW']/22.414
df['C3_Comp_Power']=(df['C3_Comp_Power_Dis_Enthpy'] - df['C3_Comp_Power_Suc_Enthpy']) / (3600)

df['LPMR_Comp_Power_Enthpy']=df['LPMR_Dis_Enthpy']-df['LPMR_Suc_Enthpy']
df['LPMR_Comp_Power']=(df['LPMR_Comp_Power_Enthpy']*df['MR_FLOW']) / (22.414*3600)

df['HPMR_Comp_Power_Enthpy']=df['HPMR_Dis_Enthpy']-df['HPMR_Suc_Enthpy']
df['HPMR_Comp_Power']=(df['HPMR_Comp_Power_Enthpy']*df['MR_FLOW']) / (22.414*3600)

#SpecPower_Enthalpy & GT_Enthalpy can be calculated now.
df['SpecPower_Enthalpy']=(df['C3_Comp_Power']+df['LPMR_Comp_Power']+df['HPMR_Comp_Power']) / df["LNG_Prod"]
df["SpecPower"]=(df["C3_GT_Power"]+ df["LPMR_GT_Power"] + df["HPMR_GT_Power"] + df["Helper_Motor_C3"] + df["Helper_Motor_LPMR"]) / (df["LNG_Prod"])

In [46]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/CGH/AID/Step2_Enthalpy Calculation/Output/SpecPower_enthpy_based_Indexed.csv",index=True)
#df.to_csv("/content/drive/MyDrive/Colab Notebooks/CGH/AID/Step2_Enthalpy Calculation/Output/SpecPower_enthpy_based.csv",index=False)